Spotify에서 id값을 이용해 artist의 데이터를 가져옴

In [ ]:
import spotipy
import argparse
import logging
import json
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import time
import random

In [ ]:
# artist_id 값을 이용해 해당 아티스트의 데이터 획득
def get_artist_info(index, dict_row, data):
    # API rate limit으로 인해 속도 제한
    time.sleep(random.randrange(25,40)/10)
    
    raw = sp.artist(dict_row['artist_id'])
    result = {
        'artist_id': raw['id'],
        'artist_name': raw['name'],
        'total_followers': raw['followers']['total'],
        'popularity': raw['popularity'],
        'genres': raw['genres'],
        'related_artists': [],
        'did_fetch_related': False
    }
    
    data[index].update(result)
    return data

In [ ]:
# artist_id를 이용해 해당 아티스트의 관련 아티스트 획득
def get_related_artists(index, dict_row, data):
    # API rate limit으로 인해 속도 제한
    time.sleep(random.randrange(25,40)/10)
    
    res = sp.artist_related_artists(dict_row['artist_id'])
    artists = res['artists'] # artists는 artist 정보를 가진 배열이 됨

    related_arr = []
    for artist in artists: # 우선 row에 relate_artists 배열을 채워줌
        related_arr.append(artist['id'])
    
    data[index].update({'related_artists': related_arr, 'did_fetch_related': True})
        
    
    for artist in artists: # artists의 길이만큼 반복
        is_exist = False
        tmp = {
            'artist_id': artist['id'],
            'artist_name': artist['name'],
            'total_followers': artist['followers']['total'],
            'popularity': artist['popularity'],
            'genres': artist['genres'],
            'related_artists': [],
            'did_fetch_related': False
        }
        for idx, d in enumerate(data): # data를 다 돌면서 이미 있는 id인지 확인
            # 중복되는 id 값이 있는지 확인
            if d['artist_id'] == artist['id']:
                # 있으면 그 row가 genres 갖고있는지 확인
                is_exist = True
                if 'genres' in d:
                    # 장르 데이터가 있으면 패스
                    break
                else:
                    # 장르 데이터가 없으면 업데이트 해줌
                    data[idx].update(tmp)  
                    break
        
        # 중복되는 id값이 없으면 json 만들어서 append
        if is_exist == False:
            data.append(tmp)
                    
    return data

In [ ]:
def fetch_data(data):
    limit = len(data)
    pbar = tqdm(data)
    
    for idx, d in enumerate(pbar):
        
        # artist detail 확인
        if 'popularity' in d:
            
            if d['popularity'] < 45:
                #break
                continue
            else:
                pbar.set_description(f"{d['artist_name']}") # 진행도 확인
                if 'did_fetch_related' in d:
                    if d['did_fetch_related'] == True:
                        continue # related까지 가져왔으면 pass

                    else:
                        if 'genres' in d:
                            # 장르가 있고 relate 없으면 relate fetch만
                            data = get_related_artists(idx, d, data)

                        else:
                            # 장르 데이터도 없고 relate가 없으면 info fetch 하고 relate fetch 둘다 진행
                            data = get_artist_info(idx, d, data)
                            data = get_related_artists(idx, d, data)

                else:
                    if 'genres' in d:
                        # 장르가 있고 relate 없으면 relate fetch만
                        data = get_related_artists(idx, d, data)

                    else:
                        # 장르 데이터도 없고 relate가 없으면 info fetch 하고 relate fetch 둘다 진행
                        data = get_artist_info(idx, d, data)
                        data = get_related_artists(idx, d, data)

                if idx >= limit:
                    print('인덱스가 너무 커져서 브레이크')
                    break
        else:
            data = get_artist_info(idx, d, data)
            data = get_related_artists(idx, d, data)

    return data

In [ ]:
file_path = "./dataset/spotify_dataset.json"

with open(file_path, 'r') as file:
    data = json.load(file)

In [ ]:
with open("./config.json", 'r') as file:
    config = json.load(file)

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=config['spotify_client_id'],
                                               client_secret=config['spotify_client_secret'],
                                               redirect_uri=config['spotify_redirect_uri'],
                                               scope=config['spotify_scope']))

In [ ]:
try:
    data = fetch_data(data)
    
except:
    print('저장')
    with open("./dataset/spotify_dataset.json", "w") as new_file:
        json.dump(data, new_file)
    fetch_data(data)

In [ ]:
with open("./dataset/spotify_dataset.json", "w") as new_file:
    json.dump(data, new_file) 